## 공시정보 - 고유번호 목록 API

DART에 등록되어있는 공시대상회사의 고유번호, 회사명, 대표자명, 종목코드, 최근변경일자를 파일로 가져옵니다.
stockcode.json에 등록되어 있는 데이터를 바탕으로 실제 증시에 상장한 공시대상회사만 선별하여 고유번호(8자리)를 corp_code.json에 저장합니다.

In [81]:
import requests
import zipfile
from io import BytesIO
import xmltodict
import json

url = "	https://opendart.fss.or.kr/api/corpCode.xml"
crtfc_key =""

r = requests.get(url, params={'crtfc_key': ""})
r.raise_for_status()
z = zipfile.ZipFile(BytesIO(r.content))
print(z.namelist())
data_xml = z.read(z.namelist()[0]).decode('utf-8')
data_odict = xmltodict.parse(data_xml)
data_dict = json.loads(json.dumps(data_odict))
data = data_dict.get('result', {}).get('list')

data = [item for item in data if item.get("stock_code") != None]

with open('stockcode.json') as r:
    stockcode = json.load(r)
    
data = [item for item in data if item.get("stock_code") in stockcode.keys()]
print(len(data))
with open("corp_code.json", "w") as w:
    json.dump(data, w, ensure_ascii=False)

['CORPCODE.xml']
2438


## 상장기업 재무정보 - 단일회사 주요계정 API

DART에서 제공하는 상장법인(금융업 제외)이 제출한 정기보고서 내에 XBRL재무제표의 주요계정과목(재무상태표, 손익계산서)의 데이터를 가져옵니다.

In [94]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'
crtfc_key = ''
payload = {'crtfc_key': crtfc_key,'corp_code': corp_code, 'bsns_year':bsns_year, 'reprt_code': reprt_code, 'fs_div':'CFS'}
r = requests.get('https://opendart.fss.or.kr/api/fnlttSinglAcnt.json', params=payload)
data_json = json.loads(r.text)
data = data_json.get('list')
df = pd.json_normalize(data)
print(df)

          rcept_no reprt_code bsns_year corp_code stock_code fs_div   fs_nm  \
0   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
1   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
2   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
3   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
4   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
5   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
6   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
7   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
8   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
9   20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
10  20200330003851      11011      2019  00126380     005930    CFS  연결재무제표   
11  20200330003851      11011      2019  00126380   

## 사업보고서 주요정보 - 증자(감자) 현황

정기보고서(사업, 분기, 반기보고서) 내에 증자(감자) 현황을 제공합니다.

In [92]:
corp_code = '00126380'
bsns_year = '2019'
reprt_code = '11011'
crtfc_key = ''
payload = {'crtfc_key': crtfc_key,'corp_code': corp_code, 'bsns_year':bsns_year, 'reprt_code': reprt_code}
r = requests.get('https://opendart.fss.or.kr/api/irdsSttus.json', params=payload)
data_json = json.loads(r.text)
data = data_json.get('list')
df = pd.json_normalize(data)
print(df)

         rcept_no corp_cls corp_code corp_name isu_dcrs_de isu_dcrs_stle  \
0  20200330003851        Y  00126380      삼성전자           -             -   

  isu_dcrs_stock_knd isu_dcrs_qy isu_dcrs_mstvdv_fval_amount  \
0                  -           -                           -   

  isu_dcrs_mstvdv_amount  
0                      -  


## 공시정보 - 공시서류원본파일

공시보고서 원본파일을 다운로드 받습니다.

In [ ]:
import requests
import zipfile
from io import BytesIO
import xmltodict
import json

url = "https://opendart.fss.or.kr/api/document.xml"

payload = {'rcept_no':'20200330003851', 'crtfc_key': ""}
r = requests.get(url, params=payload)
r.raise_for_status()

z = zipfile.ZipFile(BytesIO(r.content))
print(z.namelist())
file_list = z.namelist()
data_xml = z.read(file_list[0]).decode('cp949')


## 상장기업 재무 정보 - AWS DynamoDB에 저장하기

단일회사 주요계정 API를 사용하여 상장기업의 재무 데이터를 dynamoDB에 저장합니다.

In [ ]:
import pandas as pd

def check_value(value):
    regex = '(\d{4}).(03|06|09|12).(\d{2})'
    return re.search(regex, str(value)).group(0)

def request_single_account(payload):
    r = requests.get('https://opendart.fss.or.kr/api/fnlttSinglAcnt.json', params=payload)
    data_json = json.loads(r.text)
    if data_json.get('status') == '013':
        print(data_json.get('message'))
        return ''
    data = data_json.get('list')
    
    dart = {}
    for item in data:
        if item.get('fs_div') == 'CFS':
            account_nm = item.get('account_nm')
            thstrm_amount= item.get('thstrm_amount')
            stock_code = item.get('stock_code')
            thstrm_dt = item.get('thstrm_dt')
            dart[account_nm] = thstrm_amount.replace(',', '')
            dart['stock_code'] = stock_code
            dart['thstrm_dt'] = thstrm_dt
    if dart:
        return ''
    dart['thstrm_dt'] = check_value(dart.get('thstrm_dt'))
    
    df = pd.json_normalize(dart)
   
    result = df.to_json(orient="records")
    return json.loads(result)

def set_payload(corp_code):
    bsns_year = '2015'
    reprt_code = '11011'# 1분기: 11013, 2분기 : 11012, 3분기: 11014, 사업보고서: 11011
    fs_div = 'CFS'
    crtfc_key = ''
    return {'crtfc_key': crtfc_key, 'corp_code': corp_code, 'bsns_year':bsns_year, 'reprt_code': reprt_code, 'fs_div':fs_div}

def view_data(data):
    print(data)
    
def get_corp_code():
    with open('corp_code.json') as r:
        return json.load(r)

def do_work():
    corp_list = get_corp_code()
    for item in corp_list:
        corp_code = item.get('corp_code')
        print("start : ", corp_code)
        payload = set_payload(corp_code)
        data = request_single_account(payload)
        view_data(data)
        break
        
if __name__ == '__main__':
    do_work()